In [12]:
import urllib
import requests
import itertools
from copy import deepcopy as mcopy
from music21 import *
import threading
import time
import math

from ksxml import permute, \
    nth_permutation_indices, \
    send_m21_object, \
    write_midi_file, \
    send_transport_command, \
    stream_generator_from_chord_arrays, \
    note_list_from_note_names, \
    request_svg_creation

In [3]:
# for generating SVGs asyncronously
def stop_creating_images():
    for thread in threading.enumerate():
        if thread.isAlive() and thread.name == 'ks-image-gen':
            thread.cancel()

def create_images(streams, name_prefix, offset, current_index = 0):
    if current_index >= len(streams): return
    name = name_prefix + str(current_index + offset)
    res = request_svg_creation(streams[current_index], name)

    t = threading.Timer(0.5, create_images,
                        [streams, name_prefix, offset, current_index + 1])
    t.name = 'ks-image-gen'
    t.daemon = True
    t.start()

In [132]:
# source material
n1 = ['C3', 'D3', 'G3', 'A3']
n2 = ['C3', 'e3', 'G3', 'b3'] 
n3 = ['D3', 'E3', 'A3', 'B3']

chords_with_names = [n1, n1, n2, n2, n3, n3]

# same as above, but cycle around the circle of fifths
chord_pattern_cycle_around_circle_of_fifths = []
chord_pattern_cycle_around_circle_of_fifths_half_time = []
bass_notes_half_time = []
for note_names_in_chord, i in zip(itertools.cycle(chords_with_names), range(0, 1000)):
    transposition_amount = (math.floor(i / len(chords_with_names)) * 5) % 12
    chord_by_note_numbers = note_list_from_note_names(note_names_in_chord)
    chord_by_note_numbers = [n.pitch.midi for n in chord_by_note_numbers]
    chord_by_note_numbers_transposed = [midi_num + transposition_amount for midi_num in chord_by_note_numbers]
    chord_pattern_cycle_around_circle_of_fifths.append(chord_by_note_numbers_transposed)
    if i % 2 == 0: continue
    chord_pattern_cycle_around_circle_of_fifths_half_time.append(chord_by_note_numbers_transposed)
    bass_notes_half_time.append(chord_by_note_numbers_transposed[0])
    

In [134]:
# Create our Music21 Streams
start_index = 0
count = 1000

stream_gen = stream_generator_from_chord_arrays(chord_pattern_cycle_around_circle_of_fifths, start_index, count)
streams = [s for s in stream_gen]
full_stream = stream.Stream()
for s, i in zip(streams, range(start_index, count)):
#     s.insert(clef.BassClef())
    s.insert(meter.TimeSignature('7/8'))
    full_stream.append(s)
    
write_midi_file(full_stream.flat, '/session/default.mid')
# send_transport_command('a', 'reload')

In [167]:
chords_whole_notes = []
chords = [chord.Chord(c) for c in chord_pattern_cycle_around_circle_of_fifths_half_time]
chord_stream = stream.Stream()
bass_stream = stream.Stream()

for c in chords:
    c.quarterLength = 7
for c, i in zip(itertools.cycle(chords), range(count)):
    chord_stream.append(mcopy(c))

    # Create a stream to make the chord image from
    chord_whole_note_stream = stream.Stream()
    chord_whole_note_stream.insert(meter.TimeSignature('7/8'))
    # This chord will go in our stream
    chord_whole_note = mcopy(c)
    chord_whole_note.quarterLength = 3.5
    chord_whole_note_stream.append(chord_whole_note)
    # append the note twice
    chords_whole_notes.append(chord_whole_note_stream)
    chords_whole_notes.append(mcopy(chord_whole_note_stream))

    bass_pitch = c.bass()
    bass_note = note.Note()
    bass_note.pitch = bass_pitch
    bass_note.quarterLength = 7
    bass_stream.append(bass_note)
    


In [136]:
write_midi_file(chord_stream, '/session/chords.mid')

In [137]:
write_midi_file(bass_stream, '/session/bass.mid')

In [190]:
c = chords_whole_notes[24]
c.



False

In [99]:
# Create images from the 
create_images(streams, 'htv00-', start_index)

In [171]:
create_images(chords_whole_notes, 'htv00chord-', start_index)

In [89]:
a = request_svg_creation(streams[90], 'htv00-')
a.json()

{'path': '/score-img/htv00-.svg'}

In [170]:
stop_creating_images()

In [38]:
# Push score to browser client from a m21 stream object
send_m21_object('b', chords[0])

<Response [200]>

In [11]:
# Write a score file
write_midi_file(streams[0], '/session/default.mid')

In [194]:
# Transport control via browser
send_transport_command('a', 'imagePrefix', {'value':'htv00-'})
send_transport_command('a', 'offset', {'value':0})

send_transport_command('b', 'imagePrefix', {'value':'htv00chord-'})
send_transport_command('b', 'offset', {'value':0})

send_transport_command('a', 'play')

<Response [200]>

In [191]:
send_transport_command('a', 'stop')

<Response [200]>

In [195]:
send_transport_command('a', 'reload')

<Response [200]>

In [6]:
request_svg_creation(streams[9], 'test2')

<Response [200]>